In [1]:
import cv2
import mediapipe as mp
import numpy as np

print("MediaPipe version:", mp.__version__)
print("OpenCV version:", cv2.__version__)

MediaPipe version: 0.10.21
OpenCV version: 4.11.0


In [2]:
# # Initialize MediaPipe Pose module
mp_drawing = mp.solutions.drawing_utils  # Utility to draw landmarks
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5)

I0000 00:00:1743269899.442280 3182299 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


In [3]:
def calculate_angle(a, b, c):
    """Calculate the angle between three points using the cosine rule."""
    a = np.array(a)  # Shoulder
    b = np.array(b)  # Elbow
    c = np.array(c)  # Wrist

    # Vectors
    ba = a - b
    bc = c - b

    # Compute angle
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Ensure within valid range
    
    return np.degrees(angle)  # Convert to degrees

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# Variables for bicep curl detection
# Initialize counting variables
bicep_count = 0
arm_position = None  # Track whether the arm is "up" or "down"
elbow_threshold_up = 150  # Angle threshold for "arm extended"
elbow_threshold_down = 50  # Angle threshold for "arm flexed"

In [5]:
# Step 1: Keep asking for a valid input until provided
# choice = ""
# while choice not in ['1', '2']:
#     print("\nChoose the video source:")
#     print("1. Camera")
#     print("2. Select Video File")
#     choice = input("Enter '1' for camera or '2' for video file: ").strip()

#     if not choice:  # Handle empty input
#         print("Invalid input. Please enter '1' or '2'.")

In [6]:
# Function to open the webcam and display the feed with pose detection
def open_camera_with_pose():
    global bicep_count, arm_position, elbow_threshold_up, elbow_threshold_down
    
    """Open the webcam and display the live feed."""
    # 0 is the default camera (usually the built-in webcam)
    cap = cv2.VideoCapture(0)
    
    # Check if the video source opened successfully
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    # Read and display frames
    # Loop to keep the feed open until 'q' is pressed
    while True:
        ret, frame = cap.read() # Capture frame-by-frame
        
        if not ret:
            print("Failed to grab frame.")
            break

        # Convert the frame from BGR (OpenCV format) to RGB (MediaPipe format)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and get pose landmarks
        results = pose.process(frame_rgb)

        # If landmarks are found, draw them on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Extract important landmarks (example: shoulders and elbows)
            landmarks = results.pose_landmarks.landmark

            # Extract keypoints (Right arm)
            landmarks = results.pose_landmarks.landmark
            shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
            wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

            # Convert to pixel coordinates
            h, w, _ = frame.shape
            shoulder_coords = (int(shoulder.x * w), int(shoulder.y * h))
            elbow_coords = (int(elbow.x * w), int(elbow.y * h))
            wrist_coords = (int(wrist.x * w), int(wrist.y * h))

            # # Draw circles on keypoints
            cv2.circle(frame, shoulder_coords, 8, (255, 0, 0), -1)
            cv2.circle(frame, elbow_coords, 8, (0, 255, 0), -1)
            cv2.circle(frame, wrist_coords, 8, (0, 0, 255), -1)

            # Display coordinates
            cv2.putText(frame, f"Shoulder: {shoulder_coords}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            cv2.putText(frame, f"Elbow: {elbow_coords}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, f"Wrist: {wrist_coords}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # Calculate elbow angle
            elbow_angle = calculate_angle(shoulder_coords, elbow_coords, wrist_coords)

            # Display the angle
            cv2.putText(frame, f"Elbow Angle: {int(elbow_angle)}°", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Initialize arm position if it's the first frame
            # if arm_position is None:
            #     arm_position = "down" if elbow_angle > elbow_threshold_up else "up"
            
            # Biceps curl repetition logic
            if elbow_angle > elbow_threshold_up:
                arm_position = "down"  # Arm is extended
                
            elif elbow_angle < elbow_threshold_down and arm_position == "down":
                arm_position = "up"  # Arm is flexed
                bicep_count += 1  # Count one repetition

            # Display the repetition count
            cv2.putText(frame, f"Reps: {bicep_count}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Display the frame with the pose landmarks
        cv2.imshow("Camera Feed", frame)  # Show the live video feed

        # Exit if 'q' is pressed
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("Exiting camera feed...")
            break
            
    # Release the camera and close the OpenCV window
    cap.release()
    cv2.destroyAllWindows() # Close the window properly

# Call the function to start the camera with pose detection
open_camera_with_pose()

W0000 00:00:1743269899.502441 3182422 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743269899.514121 3182422 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743269901.200201 3182426 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-03-29 23:23:21.556 python[65515:3182299] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-29 23:23:21.556 python[65515:3182299] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Exiting camera feed...


In [7]:
# if choice == '1':
#     # Step 2: Open the webcam (Camera option)
#     # Call the function to open the camera
#     open_camera()
    
# elif choice == '2':
#     # Step 3: Open file dialog to choose a video file (Video file option)
#     root = tk.Tk()
#     root.withdraw()  # Hide the main tkinter window
#     root.call('wm', 'attributes', '.', '-topmost', True)  # Bring the dialog to the front
#     file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("MP4 files", "*.mp4"), ("All files", "*.*")])

#     if not file_path:
#         print("No file selected. Exiting.")
#         exit()

#     cap = cv2.VideoCapture(file_path)  # Open the selected video file